In [5]:
%matplotlib inline
import pystokes, autograd.numpy as np, matplotlib.pyplot as plt
from autograd import grad, elementwise_grad as egrad

In [82]:
# particle radius, self-propulsion speed, number and fluid viscosity
b, Np, eta = 1.0, 2, 0.1
PI = 3.14159265359

rij = np.array([0,1.,0])

In [65]:
## matrix elements for i=j
G01s = 1./(6*PI*eta*b)
G02a = 1./(4*PI*eta*b)
G02s = 3./(20*PI*eta*b)

In [66]:
## basics for i!=j, hard-coded version is faster (see below)

def G(xij,yij,zij, alpha,beta): #G_alpha beta
    rij = np.array([xij,yij,zij])
    r = np.linalg.norm(rij)
    return ((np.identity(3)/r + np.outer(rij,rij)/r**3)/(8*eta*PI))[alpha,beta]

def delG(xij,yij,zij, alpha,beta,gamma): #G_alpha beta, gamma = nabla_gamma G_alpha beta
    rij = np.array([xij,yij,zij])
    r = np.linalg.norm(rij)
    t1 = -np.einsum('ij,k',np.identity(3),rij)/r**3
    t2 = (np.einsum('ik,j',np.identity(3),rij) 
           + np.einsum('jk,i',np.identity(3),rij))/r**3
    t3 = -3*np.einsum('i,j,k',rij,rij,rij)/r**5
    return ((t1 + t2 + t3)/(8*eta*PI))[alpha,beta,gamma]

def lapG(xij,yij,zij, alpha,beta): # nabla^2 G_alpha beta
    rij = np.array([xij,yij,zij])
    r = np.linalg.norm(rij)
    return ((np.identity(3)/r**3 - 3*np.outer(rij,rij)/r**5)/(4*eta*PI))[alpha,
                                                                       beta]

In [67]:
epsilon = lambda i,j,k:(i-j)*(j-k)*(k-i)/2.

In [68]:
def delGauto(xij,yij,zij, alpha,beta,gamma):
    return grad(G,gamma)(xij,yij,zij, alpha,beta)

In [69]:
%%time
delG(0.,1.,3., 1,2,1) ##hard-coded

CPU times: user 671 µs, sys: 0 ns, total: 671 µs
Wall time: 616 µs


0.026422836354853946

In [70]:
%%time
delGauto(0.,1.,3., 1,2,1) ##autograd

CPU times: user 1.53 ms, sys: 376 µs, total: 1.91 ms
Wall time: 1.7 ms


0.02642283635485395

--> use hard-coded version whenever possible

In [72]:
## RBM matrix elements: G^LL for i!=j

def G1s1s(xij,yij,zij, alpha,beta):
    return (G(xij,yij,zij, alpha,beta)
            +b**2/3.*lapG(xij,yij,zij, alpha,beta))

def G1s2a(xij,yij,zij, alpha,beta):
    g1s2a=0.
    for nu in range(3):
        for eta in range(3):
            g1s2a += epsilon(beta,nu,eta)*delG(xij,yij,zij,alpha,eta,nu)
    return -0.5*b*g1s2a

def G2a1s(xij,yij,zij, alpha,beta):
    g2a1s=0
    for nu in range(3):
        for eta in range(3):
            g2a1s += epsilon(alpha,nu,eta)*delG(xij,yij,zij,eta,beta,nu)
    return 0.5*g2a1s

def G2a2a(xij,yij,zij, alpha,beta):
    g2a2a=0
    for mu in range(3):
        for kappa in range(3):
            g2a2a += epsilon(alpha,mu,kappa)*delG1s2a(xij,yij,zij, kappa,beta,mu)
    return -0.25*b*g2a2a

## auxiliary functions 
def delG1s2a(xij,yij,zij, kappa,beta,mu):
        return grad(G1s2a, mu)(xij,yij,zij, kappa,beta)

In [74]:
## G^LH and G^HL matrix elements for i!=j

def G1s2s(xij,yij,zij, alpha,kappa1,beta):
    g1s2s = 0
    g1s2s += (delG(xij,yij,zij, alpha,beta,kappa1)
              + delG(xij,yij,zij, alpha,kappa1,beta))
    g1s2s += 4*b*b/15.*(dellapG(xij,yij,zij, alpha,beta,kappa1) 
                        + dellapG(xij,yij,zij, alpha,kappa1,beta))
    return -0.5*b*g1s2s
              
def G2a2s(xij,yij,zij, alpha,kappa1,beta):
    g2a2s=0
    for nu in range(3):
        for eta in range(3):
            g2a2s += epsilon(alpha,nu,eta)*(deldelG(xij,yij,zij, eta,beta,kappa1,nu)
                                            + deldelG(xij,yij,zij, eta,kappa1,beta,nu))
    return -0.25*b*g2a2s

def G2s1s(xij,yij,zij, alpha,gamma1,beta):
    g2s1s = 0
    g2s1s += (delG(xij,yij,zij, alpha,beta,gamma1)
              + delG(xij,yij,zij, gamma1,beta,alpha))
    g2s1s += 4*b*b/15.*(dellapG(xij,yij,zij, alpha,beta,gamma1) 
                        + dellapG(xij,yij,zij, gamma1,beta,alpha))
    return 0.5*b*g2s1s

def G2s2a(xij,yij,zij, alpha,gamma1,mu):
    g2s2a=0
    for kappa1 in range(3):
        for beta in range(3):
            g2s2a += epsilon(beta,kappa1,mu)*(deldelG(xij,yij,zij, gamma1,beta,alpha,kappa1)
                                            + deldelG(xij,yij,zij, alpha,beta,gamma1,kappa1))
    return 0.25*b*b*g2s2a

## G^HH matrix element for i!=j

def G2s2s(xij,yij,zij, alpha,gamma1,kappa1,beta):
    g2s2s = (deldelG(xij,yij,zij, alpha,beta,gamma1,kappa1) 
              + deldelG(xij,yij,zij, gamma1,beta,alpha,kappa1))
    g2s2s += (deldelG(xij,yij,zij, alpha,kappa1,gamma1,beta) 
              + deldelG(xij,yij,zij, gamma1,kappa1,alpha,beta))
    g2s2s += b*b/5.*(deldellapG(xij,yij,zij, alpha,beta,gamma1,kappa1)
                     + deldellapG(xij,yij,zij, gamma1,beta,alpha,kappa1))
    g2s2s += b*b/5.*(deldellapG(xij,yij,zij, alpha,kappa1,gamma1,beta)
                     + deldellapG(xij,yij,zij, gamma1,kappa1,alpha,beta))
    return -0.25*b*b*g2s2s
              
## auxiliary functions    
def dellapG(xij,yij,zij, alpha,beta,kappa1):
    return grad(lapG, kappa1)(xij,yij,zij, alpha,beta)

def deldelG(xij,yij,zij, eta,beta,kappa1,nu):
    return grad(delG, nu)(xij,yij,zij, eta,beta,kappa1)

def deldellapG(xij,yij,zij, alpha,beta,gamma1,kappa1):
    return grad(dellapG, kappa1)(xij,yij,zij, alpha,beta,gamma1)

In [83]:
## uniaxial parametrisation
def Y2(ex,ey,ez, alpha,beta):
    e = np.array([ex,ey,ez])
    return (3*np.outer(e,e) - np.identity(3))[alpha,beta]

In [109]:
## fill tensors for tensorsolve from indices above
## RBM
def tensorG1s1s(xij,yij,zij):
    g=np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=G1s1s(xij,yij,zij, alpha,beta)
    return g

def tensorG1s2a(xij,yij,zij):
    g=np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=G1s2a(xij,yij,zij, alpha,beta)
    return g

def tensorG2a1s(xij,yij,zij):
    g=np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=G2a1s(xij,yij,zij, alpha,beta)
    return g

def tensorG2a2a(xij,yij,zij):
    g=np.zeros([3,3])
    for alpha in range(3):
        for beta in range(3):
            g[alpha,beta]=G2a2a(xij,yij,zij, alpha,beta)
    return g


## higher order
def tensorG1s2s(xij,yij,zij):
    g=np.zeros([3,3,3])
    for alpha in range(3):
        for beta in range(3):
            for gamma in range(3):
                g[alpha,beta,gamma]=G1s2s(xij,yij,zij, alpha,beta,gamma)
    return g

def tensorG2a2s(xij,yij,zij):
    g=np.zeros([3,3,3])
    for alpha in range(3):
        for beta in range(3):
            for gamma in range(3):
                g[alpha,beta,gamma]=G2a2s(xij,yij,zij, alpha,beta,gamma)
    return g

def tensorG2s1s(xij,yij,zij):
    g=np.zeros([3,3,3])
    for alpha in range(3):
        for beta in range(3):
            for gamma in range(3):
                g[alpha,beta,gamma]=G2s1s(xij,yij,zij, alpha,beta,gamma)
    return g

def tensorG2s2a(xij,yij,zij):
    g=np.zeros([3,3,3])
    for alpha in range(3):
        for beta in range(3):
            for gamma in range(3):
                g[alpha,beta,gamma]=G2s2a(xij,yij,zij, alpha,beta,gamma)
    return g

def tensorG2s2s(xij,yij,zij):
    g=np.zeros([3,3,3,3])
    for alpha in range(3):
        for beta in range(3):
            for gamma in range(3):
                for delta in range(3):
                    g[alpha,beta,gamma,delta]=G2s2s(xij,yij,zij, alpha,beta,gamma,delta)
    return g


##auxiliary tensors for uniaxial parametrisation
def tensorY2(ex,ey,ez):
    e = np.array([ex,ey,ez])
    return (3*np.outer(e,e) - np.identity(3))

def tensorG2s2sY2(xij,yij,zij,ex,ey,ez): ##LHS in lin system
    return np.einsum('ijkl,kl',tensorG2s2s(xij,yij,zij),tensorY2(ex,ey,ez))

def tensorG1s2sY2(xij,yij,zij,ex,ey,ez): 
    return np.einsum('ikl,kl',tensorG1s2s(xij,yij,zij),tensorY2(ex,ey,ez))

def tensorG2a2sY2(xij,yij,zij,ex,ey,ez): 
    return np.einsum('ikl,kl',tensorG2a2s(xij,yij,zij),tensorY2(ex,ey,ez))



In [110]:
tensorG2s2sY2(0.,0.,2., 0.,0.,1.).shape

(3, 3)

In [111]:
def vDirectSolve(v, r, F, T, e):
    for i in range(Np):
        v_ = np.zeros([3])
        for j in range(Np):
            xij = r[i]    - r[j]
            yij = r[i+Np]  - r[j+Np]
            zij = r[i+2*Np]  - r[j+2*Np]
            if i!=j:
                force  = np.array([F[j],F[j+Np], F[j+2*Np]])
                torque = np.array([T[j],T[j+Np], T[j+2*Np]])
                ex, ey, ez = e[j], e[j+Np], e[j+2*Np]
                
                lhs = tensorG2s2sY2(xij,yij,zij, ex,ey,ez)
                rhs = (np.dot(tensorG2s1s(xij,yij,zij), force) 
                       + 1./b * np.dot(tensorG2s2a(xij,yij,zij), torque))
                F02s = np.linalg.tensorsolve(lhs, rhs)
                
                v_ += (np.dot(tensorG1s1s(xij,yij,zij), force)
                      + 1./b * np.dot(tensorG1s2a(xij,yij,zij), torque)
                      - tensorG1s2sY2(xij,yij,zij, ex,ey,ez)*F02s)
            else:
                force  = np.array([F[j],F[j+Np], F[j+2*Np]])
                v_ += G01s*force
        v[i]      += v_[0]
        v[i+Np]   += v_[1]
        v[i+2*Np] += v_[2]
        
## write oDirectSolve and take into account that orientation e changes!
## Can we even give orientation? Because F2s is to be determined first...
## Might have to solve for F2s, and not for F02s
                
                

In [104]:
np.dot(np.zeros([3,3,3]),np.zeros([3])).shape

(3, 3)

In [106]:
ex, ey, ez = np.array([3,2,1])

In [107]:
ex, ey, ez

(3, 2, 1)